Although we did spelling correction in preprocess.py it was not enough. Sadly, we solve some of the misspellings but it left some unsolved and it created some other as well. So, the idea here is to try to solve the more relevant cases manually.

In [1]:
%cd /content/drive/My Drive/Colab Notebooks/regaetton_songs_nlp

/content/drive/My Drive/Colab Notebooks/regaetton_songs_nlp


In [2]:
songs_path = '/content/drive/MyDrive/Colab Notebooks/regaetton_songs_nlp/data/updated_lyrics.csv'
spanish_dict_path = '/content/drive/MyDrive/Colab Notebooks/regaetton_songs_nlp/data/spanish_dict.txt'

In [3]:
import pandas as pd
import re

from collections import Counter

In [4]:
with open(spanish_dict_path, encoding='utf-8') as f:
    spanish_dict = f.read()
    spanish_dict = spanish_dict.split('\n')
songs = pd.read_csv(songs_path)

In [6]:
songs = songs[songs.lyrics.notnull()]
songs.reset_index(drop=True, inplace=True)

# Count the occurrences of each word on all the lyrics

In [7]:
def count_ocurrences(df, spanish_dict):
    word_count = dict()
    for index, lyrics in df.lyrics.iteritems():
        
        words = lyrics.lower().split()
        for word in words:
            word_count[word] = word_count.get(word,0) + 1

    no_dict_words = dict()
    for word, count in word_count.items():
        if word not in spanish_dict:
            no_dict_words[word] = count
    
    return no_dict_words

no_dict_words = count_ocurrences(songs, spanish_dict)
no_dict_counter = Counter(no_dict_words)

In [8]:
no_dict_counter.most_common(10000)

[('pa', 4984),
 (',', 1839),
 ('aqui', 1817),
 ('the', 1808),
 ('asi', 1759),
 ('flow', 1623),
 ('you', 1284),
 ('corazon', 1278),
 ('yeah', 1205),
 ('dia', 1017),
 ('ke', 888),
 ('reggaeton', 861),
 ('estan', 792),
 ('dj', 752),
 ('hey', 737),
 ('yeh', 679),
 ('ey', 652),
 ('paul', 649),
 ('ma', 630),
 ('musica', 591),
 ('tambien', 589),
 ('perreo', 586),
 ('vamo', 561),
 ('yandel', 544),
 ('daddy', 543),
 ('ahi', 531),
 ('music', 525),
 ('despues', 515),
 ('vota', 512),
 ('ready', 505),
 ('qe', 498),
 ('my', 482),
 ('is', 477),
 ('meto', 470),
 ('besarte', 456),
 ('amarte', 440),
 ('this', 411),
 ('know', 400),
 ('on', 388),
 ('nicky', 387),
 ('girl', 381),
 ('toy', 380),
 ('vamonos', 363),
 ('lao', 362),
 ('tra', 352),
 ('we', 352),
 ('kiero', 348),
 ('remix', 338),
 ('bambino', 329),
 ('it', 315),
 ('vamosa', 313),
 ('perreando', 311),
 ('facil', 308),
 ('dejame', 304),
 ('...', 299),
 ('king', 296),
 ('prende', 292),
 ('tiran', 291),
 ('wisin', 289),
 ('ok', 288),
 ('pasion', 283)

In [9]:
df = pd.DataFrame()
df['words'] = no_dict_counter.keys()
df['count'] = no_dict_counter.values()

In [10]:
df.sort_values(by='count', ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(50)

,words,count
0,pa,4984
1,",",1839
2,aqui,1817
3,the,1808
4,asi,1759
5,flow,1623
6,you,1284
7,corazon,1278
8,yeah,1205
9,dia,1017


Most of the top errors are because they are missing the accent mark. So, we are going to solve those and the ones that are misspelled like: pa, qe, ke.

We are not going to worry about artists names, english words, and words that have a count less than 100

In [11]:
df_100 = df[df['count'] >= 100].copy()
len(df_100)

300

In [12]:
def solve_pattern(lyrics, pattern, replace):
    new_lyrics = re.sub(pattern, replace, lyrics)
    return new_lyrics


In [13]:
pattern_replace = [('pa', 'para'), ('qe', 'que'), ('ke', 'que'), ('vamo', 'vamos'), ('toy', 'estoy'),
                   ('lao', 'lado'), ('kiero', 'quiero'), ('vamosa', 'vamos a'), ('tá', 'está'), 
                   ('alante', 'adelante'), ('lante', 'delante'), ('esque', 'es que'), ('pegao', 'pegado'),
                   ('cuidao', 'cuidado'), ('migo', 'conmigo'), ('toas', 'todas')]

pattern_accent_mark = [('aqui', 'aquí'), ('asi', 'así'), ('corazon', 'corazón'), ('dia', 'día'), 
                       ('estan', 'están'), ('musica', 'música'), ('tambien', 'también'), ('ahi', 'ahí'),
                       ('despues', 'después'), ('vamonos', 'vámonos'), ('facil', 'fácil'), ('dejame', 'déjame'),
                       ('pasion', 'pasión'), ('genero', 'género'), ('cabron', 'cabrón'), ('pegate', 'pégate'),
                       ('alla', 'allá'), ('atras', 'atrás'), ('todavia', 'todavía'),
                       ('dificil', 'difícil'), ('dias', 'días'), ('razon', 'razón'), ('estes', 'estés'), 
                       ('quitate', 'quítate'), ('dejate', 'déjate'), ('oido', 'oído'), ('metele', 'métele'), 
                       ('queria', 'quería'), ('unico', 'único'), ('seras', 'serás'), ('maquina', 'máquina'), 
                       ('algun', 'algún'), ('frio', 'frío'), ('enamoro', 'enamoró'), ('hare','haré'), ('falle', 'fallé'),
                       ('jamas', 'jamás'), ('pais', 'país'), ('detras', 'detrás'), ('rapido', 'rápido'), ('habia', 'había'), 
                       ('presion', 'presión'), ('dimelo', 'dímelo'), ('numero', 'número'), ('lider', 'líder'), ('conoci', 'conocí'),
                       ('acercate', 'acércate'), ('ultima', 'última'), ('carbon', 'carbón'), ('unica', 'única'), ('lagrimas', 'lágrimas'),
                       ('olvidate', 'olvídate'), ('sere', 'seré'), ('perdon', 'perdón'), ('daria', 'daría'), ('pense', 'pensé'), 
                       ('muevete', 'muévete')]


In [22]:
#list(df_100.words)

def solve_patterns(lyrics, words):
    old, new = words
    old = r'\b{}\b'.format(old)
    new = new

    new_lyrics = re.sub(old, new, lyrics)

    return new_lyrics

for words in pattern_replace:
    songs['lyrics'] = songs.lyrics.apply(lambda x: solve_patterns(x, words))

for words in pattern_accent_mark:
    songs['lyrics'] = songs.lyrics.apply(lambda x: solve_patterns(x, words))

# Run occurrences again

In [23]:
new_no_dict_words = count_ocurrences(songs, spanish_dict)
no_dict_counter = Counter(new_no_dict_words)

In [24]:
no_dict_counter.most_common(100)

[(',', 1839),
 ('the', 1808),
 ('flow', 1623),
 ('you', 1284),
 ('yeah', 1205),
 ('pa', 931),
 ('reggaeton', 861),
 ('dj', 752),
 ('hey', 737),
 ('yeh', 679),
 ('ey', 652),
 ('paul', 649),
 ('ma', 630),
 ('perreo', 586),
 ('yandel', 544),
 ('daddy', 543),
 ('music', 525),
 ('vota', 512),
 ('ready', 505),
 ('my', 482),
 ('asi', 480),
 ('is', 477),
 ('meto', 470),
 ('besarte', 456),
 ('amarte', 440),
 ('this', 411),
 ('know', 400),
 ('on', 388),
 ('nicky', 387),
 ('girl', 381),
 ('tra', 352),
 ('we', 352),
 ('aqui', 345),
 ('pégate', 342),
 ('remix', 338),
 ('bambino', 329),
 ('it', 315),
 ('perreando', 311),
 ('...', 299),
 ('king', 296),
 ('prende', 292),
 ('tiran', 291),
 ('wisin', 289),
 ('ok', 288),
 ('jam', 279),
 ('records', 275),
 ('go', 269),
 ('booty', 266),
 ('.', 265),
 ('of', 265),
 ('cabrones', 258),
 ('tocarte', 254),
 ('hector', 253),
 ('enamore', 244),
 ('one', 238),
 ('zion', 236),
 ('comerte', 235),
 ('guille', 235),
 ('solita', 231),
 ('coming', 228),
 ('your', 226),


# Now we can save our new lyrics

In [33]:
songs.to_csv(songs_path, index=False)